In [2]:
import os 
import numpy as np

import argparse
import socket
import time
import sys

import torchvision
from torchvision import transforms
from torchvision.models import resnet50
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import copy

In [3]:
print(torch.__version__)

1.8.1+cu111


In [165]:
parser = argparse.ArgumentParser('argument for training')
# dataset
parser.add_argument('--model', type=str, default='resnet50')

# specify folder
parser.add_argument('--model_path', type=str, default='../checkpoint_0302/ours_style1_best.pth', help='path to save model')
parser.add_argument('--data_root', type=str, default='../../dataset/shape_texture_v2/', help='path to data root')

opt = parser.parse_args(args=[])

In [164]:
# trained classifier를 load해오기
opt.n_class = 20
model = resnet50(pretrained = True)
    
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, opt.n_class)

model.load_state_dict(torch.load(opt.model_path))

<All keys matched successfully>

In [6]:
for param in model.parameters():
    param.requires_grad_(False)
model = model.cuda()

In [9]:
def get_features(image, model):
    # model 레이어별로 feature 뽑아오기 (for content, gram matrix)
    x = image
    features = {}

    x = model.conv1(x)
    x = model.bn1(x)
    x = model.relu(x)
    x = model.maxpool(x)
    i = 0
    
    print(x.shape)

    for name, layer in model.layer1._modules.items():
        x = layer(x)
        i+=1
        features[f"bneck_{name}_{i}"] = x

    for name, layer in model.layer2._modules.items():
        x = layer(x)
        i+=1
        features[f"bneck_{name}_{i}"] = x

    for name, layer in model.layer3._modules.items():
        x = layer(x)
        i+=1
        features[f"bneck_{name}_{i}"] = x

    for name, layer in model.layer4._modules.items():
        x = layer(x)
        i+=1
        features[f"bneck_{name}_{i}"] = x


    return features     

In [10]:
# 생각읋 해보면, 전체 데이터셋에 대해서 할 필요는 없음 -> 데이터 한번 줄이기 (20개의 클래스별로 x 100장씩 -> 2000개씩)
# np.random.seed(10)
# import shutil
# path = '../../dataset/ours_style1_set/train'

# def save_file(class_path, save_path, filenames, start_num, end_num):
#     print(start_num)
#     print(end_num)
#     target_filenames = filenames[start_num:end_num]
#     for target_filename in target_filenames:
#         src = os.path.join(class_path, target_filename)
#         dst = os.path.join(save_path, target_filename)
#         shutil.copy(src, dst)
        
# def split_train_test():
#     for car_class in os.listdir(path):
        
#         car_class_path = os.path.join(path, car_class)
#         car_filenames = os.listdir(car_class_path)
#         train_path = os.path.join('../../dataset/ours_style1_set_100_metric_3/', car_class)

#         if not os.path.exists(train_path):
#             os.makedirs(train_path)

#         save_file(car_class_path, train_path, car_filenames, 0, 10)

# split_train_test()

In [11]:
# get datasets
test_transforms_option = transforms.Compose([
                    transforms.Resize((256, 256)),
                    transforms.ToTensor(),
                    transforms.Normalize([0.5071, 0.4867, 0.4408], [0.2675, 0.2565, 0.2761])
                ])
test_datasets = torchvision.datasets.ImageFolder(root=opt.data_root, transform = test_transforms_option)
test_loader = torch.utils.data.DataLoader(test_datasets, batch_size = 16, shuffle=False, num_workers = 4)

In [12]:
i = 0
## 레이어별로 feature 뽑아서 저장
image_features_output = dict()

with torch.no_grad() :
    
    for idx, (input, target) in enumerate(test_loader) :
        input = input.float()
        if torch.cuda.is_available():
            input = input.cuda()
            target = target.cuda()
        
        image_features = get_features(input, model)
        
        if i == 0 :
            for key, item in image_features.items():
                image_features_output[key] = item
        else :
            for key, item in image_features.items() : 
                image_features_output[key] = torch.cat([image_features_output[key], image_features[key]])
            
        i += 1

torch.Size([16, 64, 64, 64])
torch.Size([16, 64, 64, 64])
torch.Size([16, 64, 64, 64])
torch.Size([16, 64, 64, 64])
torch.Size([16, 64, 64, 64])
torch.Size([16, 64, 64, 64])
torch.Size([5, 64, 64, 64])


In [13]:
## 어떤식으로 저장되었는지 확인
for key, item in image_features_output.items():
    print(key)
    print(item.shape)
    print()

bneck_0_1
torch.Size([101, 256, 64, 64])

bneck_1_2
torch.Size([101, 256, 64, 64])

bneck_2_3
torch.Size([101, 256, 64, 64])

bneck_0_4
torch.Size([101, 512, 32, 32])

bneck_1_5
torch.Size([101, 512, 32, 32])

bneck_2_6
torch.Size([101, 512, 32, 32])

bneck_3_7
torch.Size([101, 512, 32, 32])

bneck_0_8
torch.Size([101, 1024, 16, 16])

bneck_1_9
torch.Size([101, 1024, 16, 16])

bneck_2_10
torch.Size([101, 1024, 16, 16])

bneck_3_11
torch.Size([101, 1024, 16, 16])

bneck_4_12
torch.Size([101, 1024, 16, 16])

bneck_5_13
torch.Size([101, 1024, 16, 16])

bneck_0_14
torch.Size([101, 2048, 8, 8])

bneck_1_15
torch.Size([101, 2048, 8, 8])

bneck_2_16
torch.Size([101, 2048, 8, 8])



In [14]:
# content matrix는 higher layer에서 뽑아서 지정
content_mat = image_features_output['bneck_5_13'].cpu().detach().numpy()
content_mat.shape

(101, 1024, 16, 16)

In [15]:
def gram_matrix(tensor):
    
    # input 이미지의 값을 텐서로 받아온다
    _, d, h, w = tensor.shape
    
    # multiplying the features for each channel을 위해 reshape
    tensor = tensor.reshape(d, h * w)
    
    # gram matrix
    gram = np.matmul(tensor, tensor.T)
    
    return gram

In [16]:
# lower layer에서 gram matrix를 뽑음 
i = 0

for j in range(content_mat.shape[0]) :
    
    if j % 1000 == 0 :
        print('-- check sample number : ', j)
        
    gram_tmp_output = gram_matrix(image_features_output['bneck_1_2'][j].unsqueeze(0).cpu().detach().numpy())
    gram_tmp_output = np.expand_dims(gram_tmp_output, axis=0)

    if i == 0 :
        gram_output = gram_tmp_output
    else :
        gram_output = np.concatenate((gram_output, gram_tmp_output), axis = 0 )
    i += 1

print(gram_output.shape)

-- check sample number :  0
(101, 256, 256)


In [17]:
content_mat.shape

(101, 1024, 16, 16)

## gram matrix, content matrix cluster 각각 해보기
- for outlier 제거

In [18]:
## content matrix, gram matrix 모두 2d 형태로 변형
gram_output_2d = np.reshape(gram_output, (gram_output.shape[0],(gram_output.shape[1] * gram_output.shape[1] )))
content_output_2d = np.reshape(content_mat, (content_mat.shape[0], (content_mat.shape[1] * content_mat.shape[2] * content_mat.shape[3] )))

print(gram_output_2d.shape)
print(content_output_2d.shape)

(101, 65536)
(101, 262144)


In [19]:
# 파일 이름 저장
file_nm_list = list()

for i in range(len(test_datasets.imgs)) : 
    file_nm = test_datasets.imgs[i][0]
    file_nm_list.append(file_nm)

In [22]:
import pandas as pd 
cluster_df = pd.DataFrame({'file_nm' : file_nm_list})
cluster_df.head()

,file_nm
0,../../dataset/shape_texture_v2/airplane/airpla...
1,../../dataset/shape_texture_v2/airplane/airpla...
2,../../dataset/shape_texture_v2/airplane/airpla...
3,../../dataset/shape_texture_v2/airplane/airpla...
4,../../dataset/shape_texture_v2/airplane/airpla...


In [23]:
# shape에 대한 cluster
from sklearn.cluster import KMeans

k = 10
model = KMeans(n_clusters = k, random_state = 10, max_iter = 1000)
model.fit(content_output_2d)

cluster_df['shape_cluster'] = model.fit_predict(content_output_2d)

In [168]:
cluster_df['shape_cluster'].value_counts()

1    82
0     4
5     3
9     3
6     2
8     2
4     2
7     1
3     1
2     1
Name: shape_cluster, dtype: int64

In [139]:
# outlier는 삭제해볼까
outlier_list = list()

outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 2]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 3]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 7]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 4]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 8]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 6]['file_nm'].to_list())

In [140]:
outlier_list

['../../dataset/shape_texture_v2/frog/frog_1102.png',
 '../../dataset/shape_texture_v2/deer/deer_1015.png',
 '../../dataset/shape_texture_v2/cat_stylized/cat_1765_stylized_Ink and wash painting_Ink and wash painting_13489.jpg',
 '../../dataset/shape_texture_v2/automobile_stylized/automobile_1455_stylized_Ink and wash painting_Ink and wash painting_102833.jpg',
 '../../dataset/shape_texture_v2/ship/ship_0837.png',
 '../../dataset/shape_texture_v2/airplane_stylized/airplane_0680_stylized_Ink and wash painting_Ink and wash painting_102302.jpg',
 '../../dataset/shape_texture_v2/truck_stylized/truck_1248_stylized_Ink and wash painting_Ink and wash painting_18777.jpg',
 '../../dataset/shape_texture_v2/airplane/airplane_0011.png',
 '../../dataset/shape_texture_v2/cat/cat_0131.png']

In [169]:
# shape에 대한 cluster
k = 5
model = KMeans(n_clusters = k, random_state = 10, max_iter = 1000)
model.fit(gram_output_2d)

cluster_df['texture_cluster'] = model.fit_predict(gram_output_2d)

In [170]:
cluster_df['texture_cluster'].value_counts()

1    50
3    24
0    19
2     6
4     2
Name: texture_cluster, dtype: int64

In [171]:
# outlier는 삭제해볼까
outlier_texture_list = list()
outlier_texture_list.extend(cluster_df[cluster_df['texture_cluster'] == 4]['file_nm'].to_list())
outlier_texture_list

['../../dataset/shape_texture_v2/airplane/airplane_0795.png',
 '../../dataset/shape_texture_v2/cat/cat_0659.png']

## distance 기반으로 pair
- (1) shape은 다르고 texture는 같은 pair -> 완료
- (2) shape은 같고 texture는 다른 pair 

In [166]:
# shape은 다르고 texture는 같은 것 pair (완료)
for i in range(len(cluster_df)):
    print('############################################## ')
    
    ## pair의 첫번째 img
    print(cluster_df['file_nm'][i])
    
    content_mat_i = content_mat[i]
    gram_mat_i = gram_output[i]
    
    loss_list = list()
    
    ## 전체 이미지들과 shape, texture distance 구해서 계산
    for j in range(len(cluster_df)):
        
        content_mat_j = content_mat[j]
        gram_mat_j = gram_output[j]
        
        # shape, texture loss
        loss_shape = ( content_mat_i - content_mat_j ) ** 2
        loss_texture = ( gram_mat_i - gram_mat_j ) ** 2

        # normalize (texture scale >>> shape scale)
        normalized_shape = loss_shape / np.sqrt(np.sum(loss_shape))
        normalized_texture = loss_texture / np.sqrt(np.sum(loss_texture))
        
        if i == j :
            # 자기 자신은 제외
            loss = 100000000000
        else : 
            loss = np.mean(normalized_texture) ** 2 / np.mean(normalized_shape)
        loss_list.append(loss)
    
    ## distance 제일 작은 것 index
    min_idx = np.argmin(np.array(loss_list))
    
    ## pair의 두번째 이미지
    shape_distance_min = cluster_df['file_nm'][min_idx]#[min_idx+(i+1)]
    
    
    print(shape_distance_min)
    print(np.mean(loss_list[min_idx]))
    print()

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0011.png
../../dataset/shape_texture_v2/frog/frog_1102.png
8320.596417628627

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0680.png
../../dataset/shape_texture_v2/cat/cat_1210.png
6346.03515147339

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0721.png


/tmp/ipykernel_8393/3431351782.py:24: RuntimeWarning: invalid value encountered in true_divide
  normalized_shape = loss_shape / np.sqrt(np.sum(loss_shape))
/tmp/ipykernel_8393/3431351782.py:25: RuntimeWarning: invalid value encountered in true_divide
  normalized_texture = loss_texture / np.sqrt(np.sum(loss_texture))


../../dataset/shape_texture_v2/dog/dog_2070.png
10415.133989237793

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0795.png
../../dataset/shape_texture_v2/automobile/automobile_4109.png
75524.37262889907

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_1098.png
../../dataset/shape_texture_v2/dog/dog_0238.png
8090.447497233344

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_4897.png
../../dataset/shape_texture_v2/dog/dog_0238.png
12070.543490819455

############################################## 
../../dataset/shape_texture_v2/airplane_stylized/airplane_0601_stylized_Ink and wash painting_Ink and wash painting_15469.jpg
../../dataset/shape_texture_v2/ship_stylized/ship_3167_stylized_Ink and wash painting_Ink and wash painting_100664.jpg
2113.4882455013444

############################################## 
../../dataset/shape_texture_v2/airpla

In [174]:
# shape은 같고 texture는 다른 것 pair -> 안됨
for i in range(len(cluster_df)):
    if cluster_df['file_nm'][i] in outlier_texture_list : 
        continue
    else :
    
        print('############################################## ')

        ## pair의 첫번째 img
        print(cluster_df['file_nm'][i])

        content_mat_i = content_mat[i]
        gram_mat_i = gram_output[i]

        loss_list = list()

        ## 전체 이미지들과 shape, texture distance 구해서 계산
        for j in range(len(cluster_df)):
            
            if cluster_df['file_nm'][j] in outlier_texture_list : 
                continue 
            else : 
                content_mat_j = content_mat[j]
                gram_mat_j = gram_output[j]

                # shape, texture loss
                loss_shape = ( content_mat_i - content_mat_j ) ** 2
                loss_texture = ( gram_mat_i - gram_mat_j ) ** 2

                # normalize (texture scale >>> shape scale)
                normalized_shape = loss_shape / np.sqrt(np.sum(loss_shape))
                normalized_texture = loss_texture / np.sqrt(np.sum(loss_texture))

                if i == j :
                    # 자기 자신은 제외
                    loss = 100000000000
                else : 
                    loss = np.mean(normalized_shape) **2 / np.mean(normalized_texture)
                loss_list.append(loss)
                
        if cluster_df['file_nm'][j] in outlier_texture_list : 
            continue 
        else : 
            ## distance 제일 작은 것 index
            min_idx = np.argmin(np.array(loss_list))

            ## pair의 두번째 이미지
            shape_distance_min = cluster_df['file_nm'][min_idx]#[min_idx+(i+1)]


            print(shape_distance_min)
            print(np.mean(loss_list[min_idx]))
            print()

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0011.png
../../dataset/shape_texture_v2/bird_stylized/bird_2943_stylized_Ink and wash painting_Ink and wash painting_12483.jpg
1.7832230932216052e-06

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0680.png
../../dataset/shape_texture_v2/automobile_stylized/automobile_1455_stylized_Ink and wash painting_Ink and wash painting_102833.jpg
4.7112367860550113e-07

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0721.png


/tmp/ipykernel_8393/1793145751.py:31: RuntimeWarning: invalid value encountered in true_divide
  normalized_shape = loss_shape / np.sqrt(np.sum(loss_shape))
/tmp/ipykernel_8393/1793145751.py:32: RuntimeWarning: invalid value encountered in true_divide
  normalized_texture = loss_texture / np.sqrt(np.sum(loss_texture))


../../dataset/shape_texture_v2/dog/dog_0017.png
4.2039606037459374e-07

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_1098.png
../../dataset/shape_texture_v2/automobile_stylized/automobile_1455_stylized_Ink and wash painting_Ink and wash painting_102833.jpg
3.9725533497584666e-07

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_4897.png
../../dataset/shape_texture_v2/bird/bird_0210.png
4.6090372015085863e-07

############################################## 
../../dataset/shape_texture_v2/airplane_stylized/airplane_0601_stylized_Ink and wash painting_Ink and wash painting_15469.jpg
../../dataset/shape_texture_v2/automobile_stylized/automobile_1455_stylized_Ink and wash painting_Ink and wash painting_102833.jpg
5.027463078070175e-07

############################################## 
../../dataset/shape_texture_v2/airplane_stylized/airplane_0671_stylized_Ink and wash painting_Ink and wash pain

## 클러스터 상에서의 값을 활용하는 경우
- shape 자체가 클러스터에서 제대로 안묶여서 사용하기 어려울듯

In [ ]:
## 클러스터를 사용하는 경우, 같은 shape으로 나온 것중 distance가 가장 먼것을 선정해보기
pair_nm_list = list()

for i in range(len(cluster_df)) :
    
    if i % 500 == 0 :
        print('-- check iteration: ', i)
    
    # 같은 shape으로 나온 샘플들 선정
    shape_class = cluster_df['shape_cluster'][i]
    same_shape_df = cluster_df[cluster_df['shape_cluster'] == shape_class]
    
    # 이 샘플들과의 texture distance를 구함
    loss_list = list()
    for k in same_shape_df.index : 
        loss = 0
        
        if i == k :
            continue 
        else : 
    
            ## 레이어별로 loss를 전부 계산
            for layer_nm in image_features_output.keys() :
                loss = np.mean(gram_matrix(image_features_output[layer_nm][i].unsqueeze(0).cpu().detach().numpy()) - gram_matrix(image_features_output[layer_nm][k].unsqueeze(0).cpu().detach().numpy())) **2
                loss += loss
            loss_list.append(loss)
    
    if len(loss_list) == 0:
        pair_nm_list.append('0')
    else : 
        tmp_idx = np.argmin(np.array(loss_list))        
        pair_idx = same_shape_df.index[tmp_idx]  
    
        # 제일 가까운 샘플 선정해서 df에 추가해보기
        pair_nm_list.append(cluster_df['file_nm'][pair_idx])

In [ ]:
cluster_df['pair_shape'] = pair_nm_list
cluster_df.head(10)

In [ ]:
for j in range(content_mat.shape[0]) :
    
    if j % 1000 == 0 :
        print('-- check sample number : ', j)
        
    gram_tmp_output = gram_matrix(image_features_output['bneck_1_2'][j].unsqueeze(0).cpu().detach().numpy())
    gram_tmp_output = np.expand_dims(gram_tmp_output, axis=0)

    if i == 0 :
        gram_output = gram_tmp_output
    else :
        gram_output = np.concatenate((gram_output, gram_tmp_output), axis = 0 )
    i += 1

print(gram_output.shape)

In [23]:
k = 2
model = KMeans(n_clusters = k, random_state = 10)
model.fit(gram_output_2d)

cluster_df['texture_cluster'] = model.fit_predict(gram_output_2d)

In [24]:
# 이렇게 되면 한 이미지에서 shape에 대한 cluster, texture에 대한 cluster가 나오게됨
cluster_df.head()

,file_nm,shape_cluster,texture_cluster
0,../../dataset/ours_style1_set_100_metric_3/air...,9,0
1,../../dataset/ours_style1_set_100_metric_3/air...,9,0
2,../../dataset/ours_style1_set_100_metric_3/air...,9,0
3,../../dataset/ours_style1_set_100_metric_3/air...,0,0
4,../../dataset/ours_style1_set_100_metric_3/air...,9,0


In [31]:
cluster_df.to_csv('result_texture_shape_2.csv', index =False)

In [33]:
gram_output.shape

(2000, 256, 256)

In [31]:
# 각 texture별로 돌면서, 과연 차이가 있는지 확인하기
for i in range(0, content_mat.shape[0]) :
    print('###########################################################')
    print(file_nm_list[i])
    
    shape_distance_tmp_list = list() # ith 이미지에서 제일 shape이 먼 것 확인
    texture_distance_tmp_list = list()
    
    for j in range(i+1, content_mat.shape[0]) :
        
        # 여기서는 shape을 구하고
        shape_distance = np.mean((content_mat[i] - content_mat[j]) **2) # l2 distance
        shape_distance_tmp_list.append(shape_distance)
    
        # 여기서는 texture를 구한다
        texture_distance = np.mean((gram_output[i] - gram_output[j]) **2)
        texture_distance_tmp_list.append(texture_distance)
        
    # 최종적으로는 score 확인
    score_list = list()
    
    for k in range(len(shape_distance_tmp_list)) :
        score_list.append(shape_distance_tmp_list[k] / texture_distance_tmp_list[k])
        
    ## score이 제일 큰건 shape pair
    shape_pair_idx = np.argmax(np.array(score_list)) + (i+1)
    print('> shape pair : ', file_nm_list[shape_pair_idx])
    
    ## score이 제일 작은건 texture pair
    texture_pair_idx = np.argmin(np.array(score_list)) + (i+1)
    print('> texture pair : ', file_nm_list[texture_pair_idx])
    print()

###########################################################
../../dataset/shape_texture_v2/airplane/airplane_0011.png
> shape pair :  ../../dataset/shape_texture_v2/frog/frog_1102.png
> texture pair :  ../../dataset/shape_texture_v2/airplane/airplane_0795.png

###########################################################
../../dataset/shape_texture_v2/airplane/airplane_0680.png
> shape pair :  ../../dataset/shape_texture_v2/cat/cat_1210.png
> texture pair :  ../../dataset/shape_texture_v2/airplane/airplane_0795.png

###########################################################
../../dataset/shape_texture_v2/airplane/airplane_0721.png
> shape pair :  ../../dataset/shape_texture_v2/dog/dog_2070.png
> texture pair :  ../../dataset/shape_texture_v2/airplane/airplane_0795.png

###########################################################
../../dataset/shape_texture_v2/airplane/airplane_0795.png
> shape pair :  ../../dataset/shape_texture_v2/frog/frog_1102.png
> texture pair :  ../../dataset/shape

KeyboardInterrupt: 

In [21]:
print(np.mean((gram_output[0] - gram_output[1]) **2))
print(file_nm_list[0], file_nm_list[1] )

10893.779
../../dataset/ours_style1_set_100_metric/airplane/airplane_0011.png ../../dataset/ours_style1_set_100_metric/airplane/airplane_0035.png


In [22]:
print(np.mean((gram_output[0] - gram_output[2]) **2))
print(file_nm_list[0], file_nm_list[2] )

11537.609
../../dataset/ours_style1_set_100_metric/airplane/airplane_0011.png ../../dataset/ours_style1_set_100_metric/airplane/airplane_0093.png


In [23]:
print(np.mean((gram_output[0] - gram_output[499]) **2))
print(file_nm_list[0], file_nm_list[499] )

12462.594
../../dataset/ours_style1_set_100_metric/airplane/airplane_0011.png ../../dataset/ours_style1_set_100_metric/bird/bird_4951.png


In [ ]:
# step1. 같은 shape이되 다른 texture를 가진 것 뽑아보기


In [ ]:
# step2. 같은 texture이되, 다른 shape을 가진것 뽑아보기